# animating tip positions in a single trial
Tim Tyree<br>
5.1.2023

In [1]:
from lib.my_initialization import *
import shutil

In [45]:
from lib import unwrap_and_center_xy_values
from scipy.signal import savgol_filter
from lib import ScatterPlotSnapshot
from lib import *
# printProgressBar

<function lib.utils.ProgressBar.printProgressBar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█', printEnd='\r')>

# define module

# determine chirality

In [164]:
model_name = "Fenton-Karma"
folder = f"{nb_dir}/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_10_diffcoef_0.001/trajectories"
# fn = "ic200x200.3.80_traj_sr_400_mem_0.csv"
fn = "ic200x200.3.79_traj_sr_400_mem_0.csv"

# model_name = "Luo-Rudy"
# folder = f"{nb_dir}/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/trajectories"
# fn = "ic019.33_traj_sr_600_mem_0.csv"
# os.listdir(folder)
input_dir = os.path.join(folder,fn)
os.path.exists(input_dir)

width=200
height=200

In [165]:
#load a file of tip trajectories
df = pd.read_csv(input_dir)
dt = np.around(df['t'].drop_duplicates().head(2).diff().values[-1],3)
print(f"{dt=}")

dt=0.4


In [167]:
# navg = 7
# navg = 5
# navg = 10 #4ms
# navg = 25 #10ms
navg = 15 #6ms
navg = 13 #pretty good
print(f"savgol smoothing window: {dt*navg} ms")
kwargs_savgol=dict(window_length=navg,
            polyorder=3,deriv=1,
            delta=1.0,axis=-1,mode='interp')

savgol smoothing window: 5.2 ms


In [168]:
num_frames = df.frame.drop_duplicates().shape[0]
# max_num_frames=10000
max_num_frames=2500
# max_num_frames=1000

In [170]:
#classify particle as left or right handed
#WARNING: run time ~2 minutes
# for particle,g in df.groupby('particle'):
for particle,g in df[df['frame']>=num_frames - max_num_frames].groupby('particle'):
    # unwrap
    x_values,y_values = g.sort_values(by='t')[['x','y']].values.T
    if x_values.shape[0]>=navg:
        xv,yv = unwrap_and_center_xy_values(x_values[:navg],y_values[:navg],width,height)#,jump_thresh=None,**kwargs)
#         xv,yv = unwrap_and_center_xy_values(x_values[-navg:],y_values[-navg:],width,height)#,jump_thresh=None,**kwargs)
#         xv,yv = unwrap_and_center_xy_values(x_values,y_values,width,height)#,jump_thresh=None,**kwargs)
        #compute velocities by savgol
        vx = savgol_filter(x=xv,**kwargs_savgol)
        vy = savgol_filter(x=yv,**kwargs_savgol)
        #compute chirality as mean sign of the cross-product between two adjacent velocity estimates
        cross = yv[1:]*xv[:-1] - yv[:-1]*xv[1:]
        chirality = int(np.median(np.sign(cross)))
#         chirality = int(np.sign(np.mean(np.sign(cross))))
        #record
        df.loc[df['particle']==particle,'chirality']=chirality

In [171]:
# plt.plot(g['t'].values[1:],cross)

In [172]:
df['chirality'].describe().T

count    18268.000000
mean        -0.074009
std          0.975754
min         -1.000000
25%         -1.000000
50%         -1.000000
75%          1.000000
max          1.000000
Name: chirality, dtype: float64

In [173]:
s = df['chirality']
Delta_mean_chirality = 1.96*s.std()/np.sqrt(s.count()-1.)
mean_chirality = s.mean()
print(f"{mean_chirality=:.4f} +/- {Delta_mean_chirality:.4f} (N={s.count()})")
# mean_chirality=0.0345 +/- 0.0055 (N=120993) # LR
# mean_chirality=-0.1622 +/- 0.0103 (N=34457) # FK

mean_chirality=-0.0740 +/- 0.0142 (N=18268)


In [174]:
beep(3)



0

In [175]:
saving=True
annotating=True
# message=f'{model_name} Model'
message=f'Spiral Tip Motion'
inch = 6
if saving:
    #for snapshot output
    save_folder=f"{nb_dir}/Figures/mov"
    if os.path.exists(save_folder):
        #cleanse the old snapshot folder
        shutil.rmtree(save_folder)
    os.mkdir(save_folder)

In [176]:
# for frame,g in df.groupby('frame'):
frameno=0
for frame,g in df[df['frame']>=(num_frames - max_num_frames)].groupby('frame'):
    t = np.around(dt*frame*1e-3,3)
    x_values = g['x'].values
    y_values = g['y'].values
    pid_values = g['particle'].values
    chirality_values = g['chirality'].values
    #plot tips as a scatter plot that can be saved as png
    fig=ScatterPlotSnapshot(x_values,y_values,t,width=width,height=height,
                            annotating=annotating,message=message,inch=6)
    ax = fig.get_axes()[0]
    ax.scatter(x_values[chirality_values==1],
        y_values[chirality_values==1],marker='*',c='w',s=100,alpha=1)
    ax.scatter(x_values[chirality_values==-1],
        y_values[chirality_values==-1],marker='*',c='k',s=100,alpha=1)
    if not saving:
        plt.show()
    else:
        os.chdir(save_folder)
        save_fn = f"img{frameno:07d}.png"
        plt.savefig(save_fn,dpi=720/inch, bbox_inches='tight',pad_inches=0,
                    facecolor='white');
        plt.close()
        frameno+=1
        printProgressBar(frameno,max_num_frames)

In [179]:
save_folder

'/home/timothytyree/Documents/GitHub/care/notebooks/Figures/mov'

In [180]:
# #TODO: generate a video like before,
# #concatenate the folder of .png files
# os.chdir(f'{nb_dir}/../vid/src')
# os.system(f'./concat_folder_png.sh');
beep(3)



0

In [149]:
print(f'open {nb_dir}/../vid/src')
print(f'open {nb_dir}/../vid/mov')

open /home/timothytyree/Documents/GitHub/care/notebooks/../vid/src
open /home/timothytyree/Documents/GitHub/care/notebooks/../vid/mov


In [150]:
beep(3)



0